In [2]:
%load_ext dotenv
%dotenv

ModuleNotFoundError: No module named 'dotenv'

In [3]:
import os
import json
import requests
import contextlib

import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker

import pandas as pd
import numpy as np

import re
import urllib.parse

import matplotlib.pyplot as plt

In [4]:
CONFIG_PATH = "config.json"
OUT_FILE = "traintest.pq"
VERBOSE = False

In [5]:
CONFIG = None
ENGINES = {}
TABLES = {}
BINDS = {}
SESSION = None


def config_template():
    default_conn = {
        "dialect": "postgresql",
        "host": "localhost",
        "port": 5432,
        "dbname": "INVALID",
        "schema": "public",
        "user": "INVALID",
        "passwd": "INVALID",
    }
    return {
        "dbs": {
            "login": default_conn.copy(),
            "sm": default_conn.copy(),
            "exp": default_conn.copy(),
            "ap": default_conn.copy(),
            "blogs": default_conn.copy(),
        }
    }


def get_config():
    global CONFIG
    
    if CONFIG is not None:
        return CONFIG
    if not os.path.exists(CONFIG_PATH):
        with open(CONFIG_PATH, "w") as fout:
            print(json.dumps(config_template(), indent=4, sort_keys=True), file=fout)
        raise ValueError(
            f"config file missing. new file was created at '{CONFIG_PATH}'. "
            "please correct values in file and run again")
    with open(CONFIG_PATH, "r") as fin:
        CONFIG = json.load(fin)
    config_out = {
        "dbs": {
            "login": CONFIG["dbs"]["login"].copy(),
            "sm": CONFIG["dbs"]["sm"].copy(),
            "exp": CONFIG["dbs"]["exp"].copy(),
            "ap": CONFIG["dbs"]["ap"].copy(),
            "blogs": CONFIG["dbs"]["blogs"].copy(),
        }
    }
    config_out["dbs"]["login"]["passwd"] = "..."
    config_out["dbs"]["sm"]["passwd"] = "..."
    config_out["dbs"]["exp"]["passwd"] = "..."
    config_out["dbs"]["ap"]["passwd"] = "..."
    print(f"loaded config\n{json.dumps(config_out, indent=2, sort_keys=True)}")
    return CONFIG


def get_engine(dbname):
    res = ENGINES.get(dbname)
    if res is not None:
        return res
    db = get_config()["dbs"][dbname]
    user = urllib.parse.quote_plus(db["user"])
    passwd = urllib.parse.quote_plus(db["passwd"])
    engine = sa.create_engine(
        f"{db['dialect']}://{user}:{passwd}@{db['host']}:{db['port']}/{db['dbname']}",
        echo=VERBOSE)
    engine = engine.execution_options(
        schema_translate_map={None: db['schema']})
    res = engine, sa.MetaData()
    ENGINES[dbname] = res
    return res


def get_table(dbname, tablename):
    global SESSION
    
    key = (dbname, tablename)
    res = TABLES.get(key)
    if res is not None:
        return res
    SESSION = None
    engine, metadata = get_engine(dbname)
    res = sa.Table(
        tablename,
        metadata,
        autoload_with=engine)
    TABLES[key] = res
    BINDS[res] = engine
    return res


@contextlib.contextmanager
def get_session():
    global SESSION
    
    session = SESSION
    if session is None:
        session = sessionmaker()
        session.configure(binds=BINDS)
        SESSION = session
    with session() as res:
        yield res

In [7]:
# solutions mapping tables
t_sm_pads = get_table("sm", "pads")
t_sm_tagging = get_table("sm", "tagging")

/var/folders/ff/zrdw3xx56dd1w9xbl6x_yl0c0000gn/T/ipykernel_94387/3044908452.py:85: SAWarning: Did not recognize type 'ltree' of column 'version'
  res = sa.Table(


In [17]:
with get_session() as session:
    stmt = sa.select(sa.func.count(t_sm_pads.c.id))
    stmt = stmt.where(t_sm_pads.c.status >= 2)
    sm_pad_count = int(session.execute(stmt).one()[0])
    print(sm_pad_count)

3754


In [26]:
with get_session() as session:
    stmt = sa.select(t_sm_pads.c.id, t_sm_pads.c.title, t_sm_pads.c.sections).where(t_sm_pads.c.id == 6552)
    res = session.execute(stmt).one()[2]

In [30]:
# print(json.dumps(res, indent=2, sort_keys=True))

In [31]:
def get_consents_items(sections):
    if sections is None:
        return None
    for items in sections:
        for elem in items["items"]:
            # "name": "consent",
            # "required": true,
            # "srcs": [
            #   "https://acclabplatforms.blob.core.windows.net/consent/uploads/3080f46d-45ec-4adb-99a0-25f2bf2ec578/4bdb69bca7366e7558b3bf8d0411178a.pdf"
            # ],
            # "type": "attachment"
            if elem.get('name') != "consent":
                continue
            srcs = elem.get('srcs')
            if not srcs:
                continue
            return srcs
    return None

In [33]:
consent = 0
total = 0
with get_session() as session:
    stmt = sa.select(t_sm_pads.c.id, t_sm_pads.c.title, t_sm_pads.c.sections).where(t_sm_pads.c.status >= 2)
    for row in session.execute(stmt):
        if get_consents_items(row[2]):
            consent += 1
        total += 1
consent, total

(1437, 3754)

In [37]:
consent = 0
total = 0
bad_boys = []
with get_session() as session:
    stmt = sa.select(t_sm_pads.c.id, t_sm_pads.c.title, t_sm_pads.c.sections).where(t_sm_pads.c.status >= 3)
    for row in session.execute(stmt):
        if get_consents_items(row[2]):
            consent += 1
        else:
            bad_boys.append(row[0])
            print(f"{row[0]} {row[1]}")
        total += 1
consent, total

4939 Bio-Mass Gasifier System 
5053 Mitti Cool Refrigerator
5058 Improved Bio-Stove
4950 Solar energy in the Amazon

5142 TEGOMO GAZ INSTANTANÉE 
5066 Sarai Stove
4262 Equipo de Capacitación y Apoyo
4938 Raman Deep Eco Hamam 
4940 Hydro Power Generation through Slow flow Sewage Water 
4941 Low Speed Windmill Operated Power Generator
4942 Solar Operated Paddy Reaper and Weed Cutter
4959 Power generation through sewage/slow moving water
4961 Zero Head Hydro Turbine
4963 Low cost wind mill for water pumping
4680 Carbon negative resort utilizes solar and wind energy
4243 "Public Servent Welfare" Safe Workplaces Intervention
4244 "Healthy habits programs in prisons" Intervention to Improve Chronic Health Conditions
4537 Sargazo as a biofuel in the Caribbean

4777 Solar powered water filtration in Bangladesh
4830 Turbine wind energy H-rotor
4947 A Stove in which Polythene can be used as Fuel
4948 Salan Mini Biomass Stove
4985 Modified Wood Stove, innovative windmill and other devices
4989 Fu

(1043, 1095)

In [41]:
len(bad_boys)

52

In [40]:
print(','.join(str(e) for e in bad_boys))

4939,5053,5058,4950,5142,5066,4262,4938,4940,4941,4942,4959,4961,4963,4680,4243,4244,4537,4777,4830,4947,4948,4985,4989,4990,4993,4999,5052,5057,5060,5061,5062,5064,5073,5075,5076,5077,5078,5079,5080,5081,5082,5083,5084,5085,5086,5087,5089,5090,5091,5094,5319


In [21]:
for elem in res:
    print(elem.get('type'), elem.get('title'), len(elem['items']))

section None 13


In [32]:
get_consents_items(res)

['https://acclabplatforms.blob.core.windows.net/consent/uploads/3080f46d-45ec-4adb-99a0-25f2bf2ec578/4bdb69bca7366e7558b3bf8d0411178a.pdf']

In [12]:
def has_content(item, *, only_required):
    try:
        if only_required and not item.get("required", False):
            return True
        itype = item["type"]
        if itype == "txt":
            return bool(item.get("txt", "").strip())
        if itype == "social":
            return bool(item["text"].strip())
        if itype in ("checklist", "radiolist"):
            return any((opt["checked"] for opt in item["options"]))
        if itype in ("img", "embed", "mosaic"):
            return item.get("has_content", False)
        raise ValueError(f"unknown item type: {item}")
    except Exception as e:
        raise ValueError(f"item: {item}") from e

In [13]:
def content_ratio(pid, sections, name):
    try:
        items = get_section_items(sections, name)
        if items is None:
            return 0, 0
        required = sum(has_content(item, only_required=True) for item in items) / len(items)
        extra = sum(has_content(item, only_required=False) for item in items) / len(items)
        return required, extra
    except Exception as e:
        raise ValueError(f"invalid item for pad {pid}") from e

In [14]:
content_ratio(457, res, "Design")

(1.0, 0.7857142857142857)

In [15]:
stats = {
    "id": [],
    "title": [],
    "design_required": [],
    "design_extra": [],
    "results_required": [],
    "results_extra": [],
}
with get_session() as session:
    stmt = sa.select(t_exp_pads.c.id, t_exp_pads.c.title, t_exp_pads.c.sections)
    for row in session.execute(stmt):
        design_required, design_extra = content_ratio(row[0], row[2], "design")
        results_required, results_extra = content_ratio(row[0], row[2], "results")
        stats["id"].append(row[0])
        stats["title"].append(row[1])
        stats["design_required"].append(design_required)
        stats["design_extra"].append(design_extra)
        stats["results_required"].append(results_required)
        stats["results_extra"].append(results_extra)
stats_df = pd.DataFrame(stats)
stats_df

,id,title,design_required,design_extra,results_required,results_extra
0,441,None,0.000000,0.000000,0.0,0.000
1,430,Review #1,0.000000,0.000000,0.0,0.000
2,86,,1.000000,0.000000,1.0,0.000
3,358,Incentive system for recruitment of Covid-Cham...,1.000000,1.000000,1.0,0.875
4,232,Agri Waste to Energy,1.000000,0.857143,1.0,0.125
...,...,...,...,...,...,...
308,235,Promoting Youth Entrepreneurship in Rural Areas,1.000000,0.857143,1.0,0.000
309,237,Japan Innovation challenge:,0.285714,0.000000,1.0,0.000
310,241,Marine Litter: Behavioral Insights Experiment ...,1.000000,1.000000,1.0,1.000
311,252,"Local Convergence: Promoting Agile, Adaptive, ...",1.000000,1.000000,1.0,1.000


In [16]:
stats_df["dr"] = stats_df["design_required"] > 0
stats_df["de"] = stats_df["design_extra"] > 0
stats_df["rr"] = stats_df["results_required"] > 0
stats_df["re"] = stats_df["results_extra"] > 0

In [17]:
stats_df

,id,title,design_required,design_extra,results_required,results_extra,dr,de,rr,re
0,441,None,0.000000,0.000000,0.0,0.000,False,False,False,False
1,430,Review #1,0.000000,0.000000,0.0,0.000,False,False,False,False
2,86,,1.000000,0.000000,1.0,0.000,True,False,True,False
3,358,Incentive system for recruitment of Covid-Cham...,1.000000,1.000000,1.0,0.875,True,True,True,True
4,232,Agri Waste to Energy,1.000000,0.857143,1.0,0.125,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...
308,235,Promoting Youth Entrepreneurship in Rural Areas,1.000000,0.857143,1.0,0.000,True,True,True,False
309,237,Japan Innovation challenge:,0.285714,0.000000,1.0,0.000,True,False,True,False
310,241,Marine Litter: Behavioral Insights Experiment ...,1.000000,1.000000,1.0,1.000,True,True,True,True
311,252,"Local Convergence: Promoting Agile, Adaptive, ...",1.000000,1.000000,1.0,1.000,True,True,True,True


In [18]:
pd.crosstab(stats_df["dr"], stats_df["rr"], rownames=["design"], colnames=["results"])

results,False,True
design,,
False,25,0
True,0,288


In [19]:
pd.crosstab(stats_df["de"], stats_df["re"], rownames=["design"], colnames=["results"])

results,False,True
design,,
False,59,0
True,71,183
